# Imports

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, impute
from IPython.display import display

from sklearn.dummy import DummyClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

import optuna
import statsmodels.api as sm
import scipy.stats as stats

plt.style.use('ggplot')

In [ ]:
random_state = 42
rng = np.random.RandomState(random_state)

from optuna.samplers import TPESampler
sampler = TPESampler(seed=random_state)

# Data Collection

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv', index_col=0)
X_test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv', index_col=0)

column_y = df_train.columns.difference(X_test.columns)[0]
X = df_train.drop(columns=column_y)
y = df_train[column_y]

X_proc = X.assign(missing_sum=X.isna().sum(axis=1))
X_test_proc = X_test.assign(missing_sum=X_test.isna().sum(axis=1))

del df_train, X, X_test

At first I filled the NaN with IterativeImputer and Ridge but it reduce the score. The pattern is in the NaN and XGB and LGB can use the NaN values to improve their scores. So now I let the NaN values in X

In [ ]:
# df_train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv', index_col=0)
# y = df_train['claim']
# del df_train

# X_proc = pd.read_csv('/kaggle/input/tps-092021-processing/proc.csv', index_col=0)
# X_test_proc = pd.read_csv('/kaggle/input/tps-092021-processing/proc_test.csv', index_col=0)

# X_proc = X_proc.filter(regex="^f\d+$|^missing_sum$")
# X_test_proc = X_test_proc.filter(regex="^f\d+$|^missing_sum$")

# Modeling

In [ ]:
def submit(y_pred, name):
    ys = pd.DataFrame(y_pred, index=X_test_proc.index, columns=[y.name])
    ys.to_csv(f"{name}.csv")
    return ys

## Folds

In [ ]:
# def get_folds(random_state=random_state):
#     return StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

# folds = get_folds()

In [ ]:
# fixed_folds = []

# for train_index, test_index in get_folds().split(X_proc, y):
#     fixed_folds.append((train_index, test_index))

## XGB Hyperparameter Opt.

allocate too much memory (?)

In [ ]:
xgb_params = {
    "objective":"binary:logistic",
    "eval_metric":"auc",
    'gpu_id':0,
    'tree_method':'gpu_hist',
    #"predictor": "gpu_predictor",
    "verbosity":0,
}

In [ ]:
# dtrain = xgb.DMatrix(X_proc, label=y)

In [ ]:
# eval_hist = {}
# for eta in np.geomspace(0.01, 1, 5):
#     eval_hist[eta] = xgb.cv(
#         params={'eta':eta, **xgb_params},
#         dtrain=dtrain,
#         num_boost_round=10_000,
#         early_stopping_rounds=50,
#         #folds=folds,
#         nfold=5,
#         stratified=True,
#         shuffle=True,
#         metrics='auc',
#         verbose_eval=False,
#         seed=random_state,
#     )

In [ ]:
# r_eval_hist_train, r_eval_hist_test = {}, {}
# for eta, d in eval_hist.items():
#     r_eval_hist_train[eta] = pd.Series(d['train-auc-mean'])
#     r_eval_hist_test[eta] = pd.Series(d['test-auc-mean'])

# r_eval_hist_train = pd.DataFrame(r_eval_hist_train)
# r_eval_hist_test = pd.DataFrame(r_eval_hist_test)
# r_eval_hist_test.plot(style='-')

In [ ]:
# pd.concat([r_eval_hist_test.idxmax().rename('num_boost'),
#            r_eval_hist_test.max().rename('min_test_score'),
#            r_eval_hist_train.where(r_eval_hist_test.apply(lambda x: x==x.max())).agg('sum').rename('min_train_score')], axis=1)

```
		num_boost	min_test_score	min_train_score
0.010000	3875	0.814530	0.870770
0.031623	1370	0.814381	0.875110
0.100000	360	0.813164	0.865133
0.316228	75	0.809961	0.844073
1.000000	7	0.805810	0.816480
```

In [ ]:
xgb_params['eta'] = 0.1
xgb_params['booster'] = 'gbtree'

> Your notebook tried to allocate more memory than is available. It has restarted.

In [ ]:
# def objective(trial, dtrain=dtrain):    
#     params = {
#         **xgb_params,
#         "max_depth":trial.suggest_int("max_depth", 2, 8),
#         "min_child_weight":trial.suggest_int("min_child_weight", 1, 8),
#         "colsample_bytree":trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         "subsample":trial.suggest_float("subsample", 0.2, 1.0),
#         "reg_alpha":trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
#         "reg_lambda":trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
#     }
    
#     pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "test-auc")
#     history = xgb.cv(
#         params,
#         dtrain=dtrain,
#         num_boost_round=500,
#         early_stopping_rounds=50,
#         #folds=folds,
#         nfold=5,
#         stratified=True,
#         shuffle=True,
#         metrics='auc',
#         verbose_eval=False,
#         callbacks=[pruning_callback],
#         seed=random_state,
#     )
    
#     trial.set_user_attr("n_estimators", len(history))
    
#     best_score = history["test-auc-mean"].values[-1]
#     return best_score

# pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
# xgb_study = optuna.create_study(pruner=pruner, direction="maximize", sampler=sampler)
# xgb_study.optimize(objective, n_trials=10)

In [ ]:
# print("Number of finished trials: {}".format(len(xgb_study.trials)))

# print("Best trial:")
# xgb_trial = xgb_study.best_trial

# print("  Value: {}".format(xgb_trial.value))

# print("  Params: ")
# for key, value in xgb_trial.params.items():
#     print("    {}: {}".format(key, value))
    
# print("  Number of estimators: {}".format(xgb_trial.user_attrs["n_estimators"]))

In [ ]:
# display(optuna.visualization.plot_optimization_history(xgb_study))
# display(optuna.visualization.plot_param_importances(xgb_study))

In [ ]:
xgb_params = {**xgb_params}#, **xgb_trial.params}

xgb_params['n_estimators'] = 500 #xgb_trial.user_attrs["n_estimators"] * 1.1
xgb_params['learning_rate'] = xgb_params.pop('eta')

xgbm = xgb.XGBClassifier(**xgb_params, use_label_encoder=False)
xgbm.fit(X_proc, y)
y_pred = xgbm.predict_proba(X_test_proc)[:, 1]

In [ ]:
submit(y_pred, 'xgbm')

## LightGBM Hyperparameter Opt.

In [ ]:
lgb_params = {
    "objective":"binary",
    "metric": "auc",
    "verbose":-1,
    "random_state": random_state,
}

# lgb_params["num_threads"] = 4
lgb_params = {
    "device":'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
    #"gpu_use_dp":False,
    #"max_bin":63,
**lgb_params}

### Hyperparameter Opt. part 1

In [ ]:
# dtrain = lgb.Dataset(X_proc, y)

# eval_hist = {}
# for learning_rate in np.geomspace(1e-2, 1e-1, 3):
#     eval_hist[learning_rate] = lgb.cv(
#         params={"learning_rate":learning_rate, **lgb_params},
#         train_set=dtrain,
#         num_boost_round=1_000,
#         early_stopping_rounds=100,
#         #folds=folds,
#         nfold=5,
#         stratified=True,
#         shuffle=True,
#         metrics='auc',
#         verbose_eval=False,
#     )

In [ ]:
# r_eval_hist = {}
# for eta, d in eval_hist.items():
#     r_eval_hist[eta] = pd.Series(d['auc-mean'])
    
# r_eval_hist = pd.DataFrame(r_eval_hist)
# r_eval_hist.plot()

In [ ]:
# pd.concat([r_eval_hist.idxmax().rename('num_boost'),
#            r_eval_hist.max().rename('max_test_score')], axis=1)

```
num_boost	max_test_score
0.010000	999	0.813118
0.031623	963	0.814037
0.100000	201	0.813316
```

In [ ]:
lgb_params['learning_rate'] = 0.03

### Hyperparameter Opt. part 2 with Optuna

In [ ]:
# import optuna
# from optuna.integration import lightgbm as lgb_opt

# dtrain = lgb_opt.Dataset(X_proc, y)

# study = optuna.create_study(direction='maximize', sampler=sampler)
# lgb_tuner = lgb_opt.LightGBMTunerCV(
#     params=lgb_params,
#     train_set=dtrain,
#     #folds=folds,
#     nfold=5,
#     stratified=True,
#     shuffle=True,
#     #feature_name='auto',
#     #categorical_feature=columns_proc_category,
#     return_cvbooster=True,
#     verbose_eval=-1,
#     show_progress_bar=False,
#     optuna_seed=random_state,
#     early_stopping_rounds=100,
#     num_boost_round=1_000,
#     study=study,
#     time_budget=3600*5
# )
# lgb_tuner.run()

In [ ]:
# optuna.visualization.plot_optimization_history(study)

In [ ]:
# print("Best score:", lgb_tuner.best_score)
# best_params = lgb_tuner.best_params
# print("Best params:", best_params)
# print("  Params: ")
# for key, value in best_params.items():
#     print("    {}: {}".format(key, value))

In [ ]:
# best_params = {
#     'device': "gpu",
#     'gpu_platform_id': 0,
#     'gpu_device_id': 0,
#     'objective': "binary",
#     'metric': "auc",
#     'verbose': -1,
#     'random_state': 42,
#     'learning_rate': 0.1,
#     'feature_pre_filter': False,
#     'lambda_l1': 7.747577106473336,
#     'lambda_l2': 3.404677878190547e-05,
#     'num_leaves': 7,
#     'feature_fraction': 1.0,
#     'bagging_fraction': 0.8391963650875751,
#     'bagging_freq': 5,
#     'min_child_samples': 100,
# }

In [ ]:
best_params = {
    'device': "gpu",
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
    'objective': "binary",
    'metric': "auc",
    'verbose': -1,
    'random_state': 42,
    'learning_rate': 0.03,
    'feature_pre_filter': False,
    'lambda_l1': 7.160696576959302,
    'lambda_l2': 0.05505687419638068,
    'num_leaves': 41,
    'feature_fraction': 0.984,
    'bagging_fraction': 0.8994655844810855,
    'bagging_freq': 2,
    'min_child_samples': 20, # default
}

Manual finish, the study could not run until the end, see version 3 of this notebook

In [ ]:
lgb_params = {**best_params, "n_estimators":1000}
lgbm = lgb.LGBMClassifier(**lgb_params).fit(X_proc, y)
y_pred = lgbm.predict_proba(X_test_proc)[:, 1]

In [ ]:
submit(y_pred, 'lgbm')

# Stacking

I predict y from each model by hold-out like [sklearn.ensemble.StackingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html) for proper Stacking (and avoid overfitting)

I dot not use it because I want to optimize the `final_estimator` and need the data to feed it

In [ ]:
model_dict = {
    'lgb': lgbm,
    'xgb': xgbm,
}

In [ ]:
from sklearn.model_selection import cross_val_predict

X_out_all = X_proc.copy()
X_out_test_all = X_test_proc.copy()
X_out, X_out_test = pd.DataFrame(), pd.DataFrame()

final_score = {}

for name, model in model_dict.items():
    method = 'predict_proba'
    s_train = cross_val_predict(model, X_proc, y,
                                n_jobs=1, cv=5,
                                method=method)
    s_train = s_train[:, 1]
    s_test = model.predict_proba(X_test_proc)[:, 1]
    
    final_score[name] = roc_auc_score(y, s_train)
    print(name, final_score[name])
    
    X_out[f'out_{name}'] = pd.Series(s_train, X_proc.index)
    X_out_test[f'out_{name}'] = pd.Series(s_test, X_test_proc.index)
    
X_out_all = X_out_all.join(X_out)
X_out_test_all = X_out_test_all.join(X_out_test)

In [ ]:
y_mean = X_out.mean(axis=1)
y_mean_test = X_out_test.mean(axis=1)

print("final score with mean:", roc_auc_score(y, y_mean))
submit(y_mean_test, 'final_mean')

In [ ]:
final_estimator = LogisticRegression(penalty='none').fit(X_out, y)

y_pred = final_estimator.predict_proba(X_out)[:, 1]
y_pred_test = final_estimator.predict_proba(X_out_test)[:, 1]

print("final 'training' score on training data:", roc_auc_score(y, y_pred))
submit(y_pred_test, 'final')

Our final leaderboard score should be close to the value below:

In [ ]:
r = cross_val_score(final_estimator, X_out, y, cv=5, scoring='roc_auc')

print("final score in old-out/CV:", r.mean())

# Output visualization

In [ ]:
y_pred = pd.Series(y_pred, index=y.index, name='predictions')

In [ ]:
from sklearn.metrics import roc_curve, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay

# fig, axs = plt.subplots(1, 2, figsize=(12,, 12))

cm = confusion_matrix(y, y_pred > 0.5)
cm_display = ConfusionMatrixDisplay(cm).plot()

fpr, tpr, _ = roc_curve(y, y_pred > 0.5)
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()